In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.utils import logging
import torch

In [2]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type='nf4',
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [3]:
base_model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-chat-hf', quantization_config=nf4_config)
fpf_model = AutoModelForCausalLM.from_pretrained('mesolitica/malaysian-llama2-7b-32k-instructions', quantization_config=nf4_config)
merged_model = AutoModelForCausalLM.from_pretrained('khursani8/mesollama2v2', quantization_config=nf4_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!nvidia-smi

Tue Oct 24 15:23:47 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:05:00.0  On |                  N/A |
| 58%   51C    P2             157W / 370W |  12894MiB / 24576MiB |     13%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
english_query = [
    "Translate the following English sentence to Malay: 'The cat is sitting on the table.'",
    "Translate the following Malay sentence to English: 'Saya suka makan nasi lemak.'",
    "Correct the following sentence: Me wants to go store.",
    "Is the following sentence grammatically correct? 'She neither likes coffee nor tea.'",
    "What is the meaning of the word 'philanthropy'?",
    "Explain the difference between 'affect' and 'effect.'",
    "Who is the President of the United States in 2022?",
    "What is the significance of the Mona Lisa?",
]

In [6]:
malay_query = [
    "Terjemahkan ayat berikut ke dalam Bahasa Inggeris: 'Saya suka membaca buku.'",
    "Terjemahkan ayat Bahasa Inggeris ini ke dalam Bahasa Melayu: 'The weather is nice today.'",
    "Betulkan ayat berikut: Saya suka makan nasi.",
    "Adakah ayat ini betul dari segi tatabahasa? 'Dia suka teh tetapi kopi.'",
    "Apakah maksud perkataan 'integriti'?",
    "Terangkan perbezaan antara 'hormat' dan 'takut.'",
    "Siapakah Perdana Menteri Malaysia pada tahun 2022?",
    "Apakah kepentingan Batu Caves?",

    "kwsp tu apa",
    "awat malaysia ada lembaga koko, malaysia bukan buat keluaq koko banyak pun"
]

In [7]:
from IPython.core.display import display, HTML

def display_output(query, model_name, response):
    display(HTML(f"""
    <div style="margin-bottom: 20px">
        <h3 style="color: #4CAF50;">Q: {query}</h3>
        <div style="margin-left: 20px;">
            <p><strong style="color: #2196F3;">{model_name}</strong></p>
            <pre style="background-color: #F9F9F9; border: 1px solid #ccc; padding: 10px;">{response}</pre>
        </div>
    </div>
    <hr style="border: 1px solid #E0E0E0;">
    """))
device = torch.device("cuda")
max_tok = 256

/tmp/ipykernel_17085/1274878817.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### What is the difference? Merged model will answer using language based on user question language.
### Currently this is the sweet spot where I can merge the model, need to experiment more so that all knowledge from both side available.

In [8]:
for text in english_query:
    for model in [base_model, fpf_model, merged_model]:
        msg = [
            {"role": "user", "content": text},
        ]
        encodeds = tokenizer.apply_chat_template(msg, return_tensors="pt")
        model_inputs = encodeds.to(device)
        generated_ids = model.generate(model_inputs, max_new_tokens=max_tok, do_sample=False)
        decoded = tokenizer.batch_decode(generated_ids)[0].split('[/INST]')[-1].strip()
        
        # Use the display function instead of print
        display_output(text, model.config._name_or_path, decoded)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
/home/sani/miniconda3/envs/franken/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sani/miniconda3/envs/franken/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


...

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [9]:
for text in malay_query:
    for model in [base_model, fpf_model, merged_model]:
        msg = [
            {"role": "user", "content": text},
        ]
        encodeds = tokenizer.apply_chat_template(msg, return_tensors="pt")
        model_inputs = encodeds.to(device)
        generated_ids = model.generate(model_inputs, max_new_tokens=max_tok, do_sample=False)
        decoded = tokenizer.batch_decode(generated_ids)[0].split('[/INST]')[-1].strip()
        
        # Use the display function instead of print
        display_output(text, model.config._name_or_path, decoded)

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


...

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
